## Solving the Poisson Problem on a 4 dimensional semistructured P2 Fes

In [1]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import math
import numpy as np
from numpy import exp
from methodsnm.vectorspace import *
from methodsnm.fes import *
from netgen.csg import unit_cube
from ngsolve import Mesh,VOL,specialcf

source module for methodsNM imported.


In [2]:
m = 2
T = 2
epsi = 0.1
ngmesh = Mesh(unit_cube.GenerateMesh(maxh=0.5))
mesh = UnstructuredHypertriangleMesh(T,ngmesh)
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

In [3]:
from methodsnm.forms import *
from methodsnm.formint import *
V = P2_Hypertriangle_Space(mesh)

BF = BilinearForm(V)                      
BF += LaplaceIntegral()
BF.assemble()

u_ex = lambda x: x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2
f    = lambda x:  -8.0
f = GlobalFunction(f, mesh=mesh)

LFs = LinearForm(V)
LFs += SourceIntegral(f)
LFs.assemble()

all_dofs = list(range(V.ndof))
boundary_dofs = V.boundary_dofs()
freedofs = list_diff(all_dofs, boundary_dofs)

u_h = FEFunction(V)
u_h._set_P2(u_ex,boundary=True)

res =  LFs.vector - BF.matrix @ u_h.vector 
from methodsnm.solver import solve_on_freedofs
u_h.vector += solve_on_freedofs(BF.matrix, res, freedofs)

uex = FEFunction(V)
uex._set_P2(u_ex)
res = BF.matrix @ uex.vector - LFs.vector


In [4]:
from methodsnm.forms import compute_difference_L2
l2diff = compute_difference_L2(u_h, GlobalFunction(u_ex,mesh=mesh), mesh, intorder = 3)

print("l2diff =", l2diff)

l2diff = 0.012105907985173539


In [5]:
print(V.ndof)
P1 = P1_Hypertriangle_Space(mesh)
print(P1.ndof)

652
78
